# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City_ID.1,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

city_data_df['size'] = city_data_df['Humidity']  # Size will be based on humidity

# Create the map plot
map_plot = city_data_df.hvplot.scatter(x='Lng', y='Lat', size='size', color='size', 
                                      colorbar=True, cmap='viridis', 
                                      title="City Map with Humidity as Point Size", 
                                      width=800, height=600)

# Display the map
map_plot

:Scatter   [Lng]   (Lat,size)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit the criteria (e.g., cities with latitude > 0 and humidity > 50)
filtered_data_df = city_data_df[(city_data_df['Lat'] >= 0) & (city_data_df['Humidity'] > 50)]

# Drop any rows with null values
filtered_data_df = filtered_data_df.dropna()

# Display sample data
filtered_data_df.head()

,City_ID,City_ID.1,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,size
1,1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228,100
3,3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934,77
6,6,6,albany,42.6001,-73.9662,10.36,74,29,2.10,US,1666108231,74
8,8,8,sisimiut,66.9395,-53.6735,2.72,98,100,3.79,GL,1666108235,98
11,11,11,nikolskoye,59.7035,30.7861,8.94,90,100,3.22,RU,1666108237,90


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column for "Hotel Name"
hotel_df['Hotel Name'] = None

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
1,farsund,NO,58.0948,6.8047,100,None
3,jamestown,US,42.0970,-79.2353,77,None
6,albany,US,42.6001,-73.9662,74,None
8,sisimiut,GL,66.9395,-53.6735,98,None
11,nikolskoye,RU,59.7035,30.7861,90,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 1000  # Example: search within a 1 km radius
params = {
    "apiKey": geoapify_key,  # Replace with your actual Geoapify API key
    "categories": "tourism,accommodation",  # Specify categories (e.g., hotels)
    "limit": 1,  # Only get the first result (nearest hotel)
    "lang": "en",  # Language for the search results
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"


# Display sample data
hotel_df.head()

Starting hotel search


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)